<a href="https://colab.research.google.com/github/tonyw54/GeorgeBrown/blob/main/Computer_Vision2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
import zipfile
import shutil
import os

# Re-download competition data
computer_vision_xm_path = kagglehub.competition_download('computer-vision-xm')

source_dir = '/root/.cache/kagglehub/competitions/computer-vision-xm'
destination_dir = '/content/computer-vision-xm'  # Create a folder in '/content/'

shutil.copytree(source_dir, destination_dir)

shutil.rmtree('/root/.cache/kagglehub/competitions/computer-vision-xm', ignore_errors=True)

print(f"Directory '{source_dir}' copied to '{destination_dir}' successfully.")

100%|██████████| 6.20G/6.20G [05:11<00:00, 21.4MB/s]

Extracting files...


Directory '/root/.cache/kagglehub/competitions/computer-vision-xm' copied to '/content/computer-vision-xm' successfully.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np

In [18]:
class LeafDataset(Dataset):
    def __init__(self, img_dir, df, transform=None, is_test=False):
        """
        Args:
            img_dir (str): Directory with all images
            df (pandas.DataFrame): DataFrame containing image filenames and labels
            transform: Optional transform to be applied
            is_test (bool): Whether this is test data (no labels)
        """
        self.img_dir = img_dir
        self.df = df
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get image filename
        img_name = self.df.iloc[idx, 1]  # Assuming first column is filename
        img_path = os.path.join(self.img_dir, img_name)

        # Load and convert image
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, img_name  # Return filename for test predictions
        else:
            label = self.df.iloc[idx, 2]  # Assuming second column is label
            return image, label

In [19]:
class LeafClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(LeafClassifier, self).__init__()
        # Use ResNet18 as base model
        self.model = models.resnet18(pretrained=True)

        # Freeze early layers
        for param in list(self.model.parameters())[:-4]:
            param.requires_grad = False

        # Modify final layers
        num_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [20]:
def create_data_loaders(img_dir, train_csv, test_csv, batch_size=32, val_split=0.2):
    """
    Create data loaders from CSV files
    """
    # Read CSV files
    train_df = pd.read_csv(train_csv)
    test_df = pd.read_csv(test_csv)

    # Split training data into train and validation
    train_df, val_df = train_test_split(
        train_df, test_size=val_split, random_state=42
    )

    # Define transforms
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Create datasets
    train_dataset = LeafDataset(img_dir, train_df, train_transform)
    val_dataset = LeafDataset(img_dir, val_df, val_transform)
    test_dataset = LeafDataset(img_dir, test_df, val_transform, is_test=True)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                          shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                           shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

In [21]:
def train_model(model, train_loader, val_loader, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                   patience=2, factor=0.5)

    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {avg_train_loss:.4f}')
        print(f'Val Loss: {avg_val_loss:.4f}')
        print(f'Val Accuracy: {val_accuracy:.2f}%')

        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_leaf_classifier.pth')

In [22]:
def generate_test_predictions(model, test_loader, output_file='predictions.csv'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    predictions = []
    filenames = []

    with torch.no_grad():
        for images, img_names in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            predictions.extend(predicted.cpu().numpy())
            filenames.extend(img_names)

    # Create predictions DataFrame
    predictions_df = pd.DataFrame({
        'filename': filenames,
        'prediction': predictions
    })

    # Save predictions
    predictions_df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

In [23]:
# Usage example
if __name__ == "__main__":
    # Initialize model and data loaders
    model = LeafClassifier()

    # Paths to your data
    IMG_DIR = '/content/computer-vision-xm/images/kaggle/working/Reorganized_Data/images'
    TRAIN_CSV = '/content/computer-vision-xm/train.csv'
    TEST_CSV = '/content/computer-vision-xm/test.csv'

    # Create data loaders
    train_loader, val_loader, test_loader = create_data_loaders(
        IMG_DIR, TRAIN_CSV, TEST_CSV, batch_size=32
    )

    # Train the model
    train_model(model, train_loader, val_loader)

    # Generate predictions for test set
    generate_test_predictions(model, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10]
Train Loss: 0.4938
Val Loss: 0.3730
Val Accuracy: 82.01%
Epoch [2/10]
Train Loss: 0.3915
Val Loss: 0.3054
Val Accuracy: 88.35%
Epoch [3/10]
Train Loss: 0.3594
Val Loss: 0.3154
Val Accuracy: 85.25%
Epoch [4/10]
Train Loss: 0.3436
Val Loss: 0.2877
Val Accuracy: 88.94%
Epoch [5/10]
Train Loss: 0.3406
Val Loss: 0.2860
Val Accuracy: 87.91%
Epoch [6/10]
Train Loss: 0.3101
Val Loss: 0.3121
Val Accuracy: 85.69%
Epoch [7/10]
Train Loss: 0.3157
Val Loss: 0.2887
Val Accuracy: 87.91%
Epoch [8/10]
Train Loss: 0.3070
Val Loss: 0.2846
Val Accuracy: 86.73%
Epoch [9/10]
Train Loss: 0.3024
Val Loss: 0.3335
Val Accuracy: 86.28%
Epoch [10/10]
Train Loss: 0.3000
Val Loss: 0.2771
Val Accuracy: 88.35%
Predictions saved to predictions.csv


In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Make predictions on the test dataset
model.eval()
predictions = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a DataFrame with IDs and predictions
submission_df = pd.DataFrame({
    'Images': test_dataset.labels_df['Images'],
    'Labels': predictions
})
submission_df.to_csv('submission.csv', index=False)
print('Submission file was created.')

NameError: name 'model' is not defined